# *Agents and Tools with Government APIs*

[Unleashing the Power of Langchain Agents and Tools with Government APIs](https://medium.com/prompt-engineering/unleashing-the-power-of-langchain-agents-and-tools-fe4567e56187)

In [ ]:
!pip install --quiet langchain langchain_openai langchain_community langchain_core langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI()

In [ ]:
from langchain.utilities import TextRequestsWrapper
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
import requests

# The code shows how to create custom tools by Subclassing the BaseTool class.

- search_police_force: This tool searches for police forces based on a query and returns the matching forces.

In [ ]:
def get_police_forces():
  url = "https://data.police.uk/api/forces"
  response = requests.get(url)

  if response.status_code == 200:
      return response.json()
  else:
      return None

In [ ]:
def search_police_forces(query):
  police_forces = get_police_forces() # Call the url

  if not police_forces:
      print("Error: Unable to fetch police forces")
      return

  matching_forces = []

  for force in police_forces:
      if query.lower() in force["name"].lower():
          matching_forces.append(force)

  return matching_forces

In [ ]:
# Example usage
force_id = "devon"
police_force_data = search_police_forces(force_id) # Call the baseclass
print (police_force_data)

[{'id': 'devon-and-cornwall', 'name': 'Devon & Cornwall Police'}]


In [ ]:
class search_police_force(BaseTool):
  name = "search_police_force"
  description =  "useful for when you need to answer questions about police forces, such as the id Unique force identifier and Police Force name"

  def _run(self, query: str) -> str:
    "Use the tool. "
    return search_police_forces(query)

  async def _arun(self, query: str) -> str:
    "Use the tool asynchronoulsy"
    raise NotImplementedError("API does not support async")

* get_police_force_details: This tool retrieves information about a specific police force, such as its website, contact information, and unique force identifier.

In [ ]:
def get_police_force_id(force_id):
    url = f"https://data.police.uk/api/forces/{force_id}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None

In [ ]:
# Example usage
force_id = "devon-and-cornwall"
police_force_data = get_police_force_id(force_id)
print (police_force_data)

{'description': '<p>Devon &amp; Cornwall Police covers the largest geographical police area in England, extending 180 miles from the Dorset and Somerset borders in the east to the Isles of Scilly in the west.<br />\t\t<br />\t\tTo give some impression of the scale of the area, police headquarters in Exeter is actually nearer to London than the furthest extremity of the Force.<br />\t\t<br />\t\tPolicing an area of this size offers many challenges, with the needs of both rural and urban communities continually at the forefront of the Force\'s approach to tackling crime.</p>\n\n<p>\t\tThe Force aim is:<br />\t\t"By working in partnership to: bring about safer communities; reduce disorder, crime and the fear of crime; and contribute to the delivery of justice in a way which secures and maintains public confidence."</p>', 'url': 'http://www.devon-cornwall.police.uk', 'engagement_methods': [{'url': 'http://www.facebook.com/pages/Exeter-United-Kingdom/Devon-Cornwall-Police/151265442987', 'ty

In [ ]:
class get_police_force_details(BaseTool):
    name = "get_police_force_details"
    description = "useful for when you need to answer questions about a specific police force such as the website, ways to keep informed, telephone number, Unique force identifier or Force name"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return get_police_force_id(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

* get_police_force_peoples: This tool obtains Senior officer bio information for a given police force.


In [ ]:
def get_police_force_people(force_id):
    url = f"https://data.police.uk/api/forces/{force_id}/people"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None

In [ ]:
# Example usage
force_id = "leicestershire"
police_force_data = get_police_force_people(force_id)
print (police_force_data)

[{'bio': '<p>Rob has served with Leicestershire Police for 25 years, beginning his policing career here in 1993.</p>\n\n<p>In 2008 he was appointed as the head of the force’s Community Safety Bureau, which was responsible for the implementation of neighbourhood policing, tackling anti-social behaviour and offender management.</p>\n\n<p>His next role, from 2009 to 2014, was as Chief Superintendent for the Leicester City Basic Command Unit. He led local policing for the area and worked with communities during two large scale English Defence League and Unite Against Fascism demonstrations. He was instrumental in the force’s response to disorder during August 2012.</p>\n\n<p>Throughout 2014 and 2015 he was head of business change and developed Blueprint2020 – a programme designed to decide how to make significant savings and take the force into a new era of policing.</p>\n\n<p>His next step was to become Temporary Assistant Chief Constable, working across Leicestershire, Northamptonshire a

In [ ]:
class get_police_force_peoples(BaseTool):
    name = "get_police_force_peoples"
    description = "useful for when you need to answer questions about Senior officer bio"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return get_police_force_people(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

* get_police_neighbourhoods: This tool fetches the names of police force neighbourhoods based on a given force ID.

In [ ]:
def get_neighbourhoods(force_id):
    url = f"https://data.police.uk/api/{force_id}/neighbourhoods"
    cleaned_url = url.replace("'", "")
    response = requests.get(cleaned_url)
    neighbourhoods = response.json()

    if response.status_code == 200:
        names = [entry['name'] for entry in neighbourhoods]
        names_string = ', '.join(names)
        return (names_string)
    else:
        return None

In [ ]:
# Example usage
force_id = "\'devon-and-cornwall\'"
neighbourhood_data = get_neighbourhoods(force_id)
print (neighbourhood_data)

Camborne North, Camborne South, Camborne West, Illogan North, Illogan South, Redruth North, Redruth South &amp; St Day &amp; Lanner, Helston Town, Lizard, Porthleven &amp; Helston North, Penzance Rural East, Penzance Rural West, Penzance Town East, Penzance Town West, St Just, Pendeen &amp; Sennen, Carbis Bay, Copperhouse &amp; Gwinear, Hayle Foundry, Isles of Scilly, St Ives Town, Arwenack &amp; Boslowick, Penwerris &amp; Trescobeas, Newquay Town, St Columb Major, Grampound &amp; Mevagissey, Lostwithiel &amp; Fowey, St Austell Town, St Blazey &amp; Tywardreath, St Stephen &amp; St Dennis, Treverbyn &amp; Roche, Boscawen, Moresk &amp; Tregolls, Mount Hawke &amp; St Agnes, Perranporth, Roseland &amp; Probus &amp; Trispen &amp; St Erme, Treliske Hospital, Bodmin Rural, Bodmin St Marys, Bodmin St Petroc, Padstow &amp; St Merryn, Wadebridge Town, Rock, Polzeath &amp; Port Isaac, Bude Rural, Bude Town, Camelford &amp; Tintagel, Launceston, Launceston Rural, Liskeard North, Liskeard Rural, L

In [ ]:
class get_police_neighbourhoods(BaseTool):
    name = "get_police_neighbourhoods"
    description = "useful for when you need to answer questions about the name for the police force neighbourhood. You will need to use GetPoliceforce first to get the force name for the API"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return get_neighbourhoods(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

* locate_police_neighbourhood: This tool locates a police neighbourhood using latitude and longitude coordinates.


In [ ]:
def locate_neighbourhood(latlong):

    url = f"https://data.police.uk/api/locate-neighbourhood?q={latlong}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None

In [ ]:
# Example usage
latlong = "50.3656 ,-4.1423"
neighbourhood_data = locate_neighbourhood(latlong)
print (neighbourhood_data)

{'force': 'devon-and-cornwall', 'neighbourhood': 'PLY.4028'}


In [ ]:
class locate_police_neighbourhood(BaseTool):
    name = "locate_police_neighbourhood"
    description = "useful for when you need to answer questions about the location of a police neighbourhood using lattitude, longitude numbers. Pass lattitude and longitude as a string"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return locate_neighbourhood(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Create Tools

In [ ]:
# Load the tool configs that are needed.
tools = [
    Tool(
        name="SearchPoliceForce",
        func=search_police_force.run,
        description="useful for when you need to answer questions about police forces, such as the id	Unique force identifier and Police Force name"
    ),
    Tool(
        name="GetPoliceforceDetails",
        func=get_police_force_details.run,
        description="useful for when you need to answer questions about a specific police force such as the website, ways to keep informed, telephone number, Unique force identifier or Force name"
    ),
    Tool(
        name="GetPoliceforcePeople",
        func=get_police_force_peoples.run,
        description="useful for when you need to answer questions about Senior officer bio"
    ),
        Tool(
        name="GetPoliceforceNeighbourhood",
        func=get_police_neighbourhoods.run,
        description="useful for when you need to answer questions about the name for the police force neighbourhood. You will need to use GetPoliceforceDetails first to get the force name for the API"
    ),
    Tool(
        name="LocatePoliceNeighbourhood",
        func=locate_police_neighbourhood.run,
        description="useful for when you need to answer questions about the location of a police neighbourhood using lattitude, longitude numbers. Pass lattitude and longitude as a string"
    )
]

In [ ]:
tools = [search_police_force(), get_police_force_details(), get_police_neighbourhoods(), locate_police_neighbourhood()]

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
# Locate police force by lat, long [Still Work in progress]
agent.run("Get the police chief for latitude 50.3656, longitude -4.1423")



> Entering new AgentExecutor chain...
 I need the police force name by using the locate_police_neighbourhood function
Action: locate_police_neighbourhood
Action Input: "50.3656, -4.1423"
Observation: {'force': 'devon-and-cornwall', 'neighbourhood': 'PLY.4028'}
Thought: I need the police force name
Action: get_police_force_details
Action Input: "devon-and-cornwall"
Observation: {'description': '<p>Devon &amp; Cornwall Police covers the largest geographical police area in England, extending 180 miles from the Dorset and Somerset borders in the east to the Isles of Scilly in the west.<br />\t\t<br />\t\tTo give some impression of the scale of the area, police headquarters in Exeter is actually nearer to London than the furthest extremity of the Force.<br />\t\t<br />\t\tPolicing an area of this size offers many challenges, with the needs of both rural and urban communities continually at the forefront of the Force\'s approach to tackling crime.</p>\n\n<p>\t\tThe Force aim is:<br />\t\t"

'Unfortunately, the police chief name for Devon & Cornwall Police is not available.'